In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import sklearn 
import seaborn as sns


writer_r = pd.ExcelWriter(r'./RF.xlsx')

writer_t = pd.ExcelWriter(r'./RF_Feature_Ranking.xlsx')

In [83]:
number = 14
data_path = r'./data.xlsx'
sheet_names= pd.ExcelFile(data_path).sheet_names
sheet_name = sheet_names[number]
print(sheet_name)

data = pd.read_excel(data_path,skiprows=0,sheet_name=sheet_name )
data= data.dropna()
# print(data.columns)

features=data[data.columns[2:]]
labels=data[data.columns[1]]

# Standardization
scaler=preprocessing.StandardScaler().fit(features)
features_=scaler.transform(features)
features_=pd.DataFrame(features_,columns=features.columns)
# print(features_)
print(features_.shape)

# Shuffle data order
indices = np.arange(features_.shape[0])
np.random.shuffle(indices)

features_ = features_.iloc[indices]
labels = labels.iloc[indices]


根系day20
(334, 35)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, r2_score, mean_squared_error
name='RF'
# Define RMSE
def rmse_scorer(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
rmse = make_scorer(rmse_scorer, greater_is_better=False)


# Define parameter value range
param_grid = {
    'n_estimators': np.arange(1,100,3),
    'max_depth': [8,10,12],
}

# start prediction
rf = RandomForestRegressor(max_features='auto',random_state=1234)

scoring = {'R2': 'r2', 'RMSE': rmse}
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=10, scoring=scoring,refit='R2',n_jobs=-1, verbose=2)
grid_search.fit(features_, labels)

# Print best parameters and results
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation R2 score: {:.4f}".format(grid_search.cv_results_['mean_test_R2'].max()))
print("Corresponding RMSE for best R2:", grid_search.cv_results_['mean_test_RMSE'][grid_search.cv_results_['mean_test_R2'].argmax()])

In [ ]:
# Save parameters
import pandas as pd
best_params = grid_search.best_params_
best_r2_score = grid_search.cv_results_['mean_test_R2'].max()
best_rmse_score = grid_search.cv_results_['mean_test_RMSE'][grid_search.cv_results_['mean_test_R2'].argmax()]
data = {'name':name,
    "n_estimators": [best_params['n_estimators']],
    "max_depth": [best_params['max_depth']],
    "Best R2 Score": [best_r2_score],
    "Corresponding RMSE": [best_rmse_score]
}

results_df = pd.DataFrame(data)
results_df.to_excel(writer_r,sheet_name=sheet_name)
print(results_df)

In [ ]:
# Save model
import os
import pickle
path = r'./RF'
path_model = path + '/' + sheet_name + '.pkl'
try:
    with open(path_model, 'wb') as f:
        pickle.dump(grid_search.best_estimator_, f) 
    print("Model has been successfully saved to:", path_model)
except Exception as e:
    print("Error saving model:", e)

In [ ]:
best_model = grid_search.best_estimator_

# Retrieve and rank important features
importance = pd.Series(best_model.feature_importances_, index = features_.columns)
importance.sort_values().plot(kind='barh')
plt.rcParams['font.sans-serif'] = ['SimHei']  
plt.rcParams['axes.unicode_minus'] = False 
plt.show()

sorted_features = sorted(zip(importance,features_.columns),reverse=True)
sorted_features=pd.DataFrame(sorted_features)
sorted_features.to_excel(writer_t,index=0,sheet_name=sheet_name)
print(sorted_features)


In [88]:
writer_r.save()
writer_t.save()
writer_r.close()
writer_t.close()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20148\2608853572.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer_r.save()
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20148\2608853572.py:3: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer_t.save()
